In [11]:
# todo
# 修改info代码 gflops task.py self.info()
# bbox融合可以作为 冗余备份
# https://blog.csdn.net/weixin_43694096/article/details/126354660

# Utils函数

In [12]:
import os
import glob
import pandas as pd


def read_single_map(path_csv, type_map='5095'):
    # 读取一个csv文件中的map50 or map5095
    df = pd.read_csv(path_csv)
    if type_map == '5095':
        row = df.iloc[0:1].copy()
    elif type_map == '50':
        row = df.iloc[1:2].copy()
    
    # 读取参数量 计算量 帧率
    if df.shape[0]>12:
        row['param'] = df.iloc[12,1]
        row['flops'] = df.iloc[13,1]
        row['delay'] = df.iloc[14,1]
        row['fps'] = df.iloc[15,1:].mean().round(1)
    
    # metrics替换成models
    row.rename(columns={'metrics': 'models'}, inplace=True)
    filename = os.path.basename(path_csv)
    model_name = filename.split(f'metrics_')[1].split('.csv')[0]
    row['models'] = model_name
    return row

def read_sml_map(path_csv):
    # 读取一个csv文件中的map s m l
    df = pd.read_csv(path_csv, index_col='metrics')
    rows = df.iloc[3:6,:].copy()
    return rows

def read_all_single_map(path_csv, type_map='5095'):
    # 从目录读取所有csv内的metrics并整合成df
    all_path = glob.glob(f'{path_csv}/*.csv')
    metrics = [read_single_map(path, type_map) for path in all_path]
    return metrics

def set_metrics(metrics, sort=True, is_finetune=False):
    # 把metrics_list转化为df
    metrics = pd.concat(metrics, axis=0)
    metrics.set_index('models', inplace=True)

    # 按不同任务类型排序
    if sort:
        if is_finetune:
            metrics.sort_values(by='test_real_edge', ascending=True, inplace=True)
        else:
            metrics.sort_values(by='test_real', ascending=True, inplace=True)
    return metrics

def display_metircs(metrics, basemodel='', is_finetune=False, filename=''):
    # 可视化metrics df

    # 仅保留比basemodel优异的行
    if basemodel:
        base_index = metrics.index.get_loc(basemodel)
        metrics = metrics.iloc[base_index:]
    
    # style条件
    all_datasets = ['test_synth', 'test_real_nominal', 'test_real_edge', 'test_real', 'test']
    all_others = ['param', 'flops', 'delay', 'fps']

    # 应用style
    bar_cols = all_datasets
    if is_finetune:
        bar_cols = ['test_real_edge']

    # metrics[all_datasets] = metrics[all_datasets] * 100

    if len(metrics) > 1:
        if 'param' in metrics.columns.tolist():
            display(metrics.style \
                .bar(align='left', subset=bar_cols) \
                .format("{:.3f}", subset=all_datasets) \
                .format("{:.3f}", subset=all_others[:2]) \
                .format("{:.1f}", subset=all_others[-2:]) \
                .highlight_min(color='pink', subset=all_others[:3]) \
                .highlight_max(color='pink', subset=all_others[-1])
            )
        else:
            display(metrics.style.bar(align='left', subset=all_datasets).format("{:.3f}", subset=all_datasets))
    else:
        display(metrics)

    # 保存df
    metrics_for_save = metrics.copy()
    metrics_for_save.to_csv(f'papers/metircs/{filename}.csv')
    
def format_and_combine(df2, improve_percent):
    # 合并df2和df2相对于df1的提升信息

    # style条件
    all_datasets = ['test_synth', 'test_real_nominal', 'test_real_edge', 'test_real', 'test']
    all_others = ['param', 'flops', 'delay', 'fps']

    improve_percent_formatted = improve_percent.applymap(lambda x: f"({'+' if x >= 0 else '-'}{abs(x):04.1f}%)")  # 补0 追加sign和()

    if 'param' not in df2.columns.tolist():
        df2[all_datasets] = df2[all_datasets].applymap(lambda x: f"{x:.3f}")
    else:
        df2[all_datasets+all_others[:2]] = df2[all_datasets+all_others[:2]].applymap(lambda x: f'{x:.3f}')
        df2[all_others[-2:]] = df2[all_others[-2:]].applymap(lambda x: f'{x:.1f}')

    # 合并
    df_concatenated = df2 + " " + improve_percent_formatted
    return df_concatenated

def display_single_improve(metrics, basemodel, improved_model, filename=''):
    print(f"{improved_model}相较于{basemodel}的改进")

    # 取出model_name所在行的series
    df1 = metrics.loc[[basemodel]].rename(index={basemodel: 'map'})  # series
    df2 = metrics.loc[[improved_model]].rename(index={improved_model: 'map'})

    # 计算提升百分比
    better = ((df2-df1)/df1*100).round(1)
    
    all_datasets = ['test_synth', 'test_real_nominal', 'test_real_edge', 'test_real', 'test']
    all_others = ['param', 'flops', 'delay', 'fps']

    # df2数据+对df1的百分比
    df2_formatted = format_and_combine(df2, better)

    # df1格式化
    col3 = all_datasets + all_others[:2] if 'param' in df1.columns.tolist() else []
    col1 = all_others[-1] if 'param' in df1.columns.tolist() else []
    df1[col3] = df1[col3].applymap(lambda x: f"{x:.3f}")
    df1[col1] = df1[col1].apply(lambda x: f"{x:.1f}")

    # 合并
    df1_df2formatted = pd.concat([df1, df2_formatted]) \
                            .replace('nan (-0nan%)', '-') \
                            .replace('-1.000', '-') \
                            .replace('-1.000 (+00.0%)', '-')

    display(df1_df2formatted)
    # 保存df
    df1_df2formatted.to_csv(f'papers/metircs/{filename}.csv')

def display_sml_improve(basemodel, improved_model, filename=''):

    # 读取sml map
    df1 = read_sml_map(path_csv=f'results/tables/metrics/metrics_{basemodel}.csv')
    df2 = read_sml_map(path_csv=f'results/tables/metrics/metrics_{improved_model}.csv')

    # 计算提升百分比
    better = ((df2-df1)/df1*100).round(1)

    all_datasets = ['test_synth', 'test_real_nominal', 'test_real_edge', 'test_real', 'test']
    all_others = ['param', 'flops', 'delay', 'fps']

    # df2数据+对df1的百分比
    df2_formatted = format_and_combine(df2, better)

    #  df1格式化
    if 'param' in df1.columns.tolist():
        col3 = all_datasets + all_others[:2]
        col1 = all_others[-1]
        df1[col3] = df1[col3].applymap(lambda x: f"{x:.3f}")
        df1[col1] = df1[col1].apply(lambda x: f"{x:.1f}")
    else:
        col3 = all_datasets
        df1[col3] = df1[col3].applymap(lambda x: f"{x:.3f}")

    # 合并df1 df2
    df1_df2formatted = pd.concat([df1, df2_formatted]) \
                            .replace('nan (-0nan%)', '-') \
                            .replace('-1.000', '-') \
                            .replace('-1.000 (+00.0%)', '-')
    display(df1_df2formatted)

    # 保存df
    df1_df2formatted_for_save = df1_df2formatted.copy()
    df1_df2formatted_for_save.to_csv(f'papers/metircs/{filename}.csv')

! rm -r papers/csv
! mkdir -p papers/csv

# BARS and MSFS

In [13]:
metrics = []
metrics.append(read_single_map(f'results/tables/metrics/other_dataset/metrics_finetune_bars_runway_val_test_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640.csv', type_map='50'))
metrics.append(read_single_map(f'results/tables/metrics/other_dataset/metrics_finetune_bars_runway_val_test_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640.csv', type_map='5095'))
display(pd.concat(metrics, axis=0))

metrics = []
metrics.append(read_single_map(f'results/tables/metrics/other_dataset/metrics_finetune_msfs_runway_test_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640.csv', type_map='50'))
metrics.append(read_single_map(f'results/tables/metrics/other_dataset/metrics_finetune_msfs_runway_test_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640.csv', type_map='5095'))
display(pd.concat(metrics, axis=0))

,models,bars_runway_val_test
1,finetune_bars_runway_val_test_yolov8n_lska_bif...,0.989
0,finetune_bars_runway_val_test_yolov8n_lska_bif...,0.964


,models,msfs_runway_test
1,finetune_msfs_runway_test_yolov8n_lska_bifpn_E...,0.946
0,finetune_msfs_runway_test_yolov8n_lska_bifpn_E...,0.794


# baseline分析

In [14]:
metrics = read_all_single_map('results/tables/metrics/baseline')
metrics.append(read_single_map(f'results/tables/metrics/metrics_base_yolov8n_640_640.csv'))
metrics.append(read_single_map(f'results/tables/metrics/metrics_merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640.csv'))
metrics = set_metrics(metrics)
metrics = metrics[metrics.index != 'baseline_centernet_512']
metrics = metrics[metrics.index != 'baseline_retinanet_1333']
display_metircs(metrics, filename='baseline')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
baseline_yolov3_608,0.684,0.492,0.233,0.448,0.581,61.524,58.861,12.4,83.9
baseline_yolov6n_640,0.722,0.524,0.219,0.473,0.613,4.301,5.495,14.7,70.9
baseline_ssd_512,0.670,0.530,0.244,0.482,0.588,24.386,87.541,14.9,65.8
baseline_yolov7t_640,0.687,0.524,0.316,0.488,0.599,6.015,6.551,9.4,103.6
baseline_yolov5n_640,0.706,0.540,0.302,0.499,0.614,1.765,2.088,10.2,100.1
baseline_faster_rcnn_1333,0.719,0.566,0.286,0.517,0.630,41.348,158.276,28.4,32.7
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,0.760,0.611,0.413,0.578,0.678,3.106,8.308,8.4,125.3


# 模块对比

## 注意力

In [15]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|att_yolov8n')]
metrics = metrics[metrics.index != 'att_yolov8n_ese_640_640']
metrics = metrics[metrics.index != 'att_yolov8n_sa_640_640']
metrics = metrics[metrics.index != 'att_yolov8n_ta_640_640']
display_metircs(metrics, filename='att')
display_single_improve(metrics, 'base_yolov8n_640_640', 'att_yolov8n_lska_640_640', filename='improve_lska_vs_base')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
att_yolov8n_ema_640_640,0.754,0.561,0.323,0.521,0.651,3.012,8.215,8.6,122.0
att_yolov8n_cbam_640_640,0.758,0.568,0.340,0.528,0.656,3.077,8.247,7.8,132.8
att_yolov8n_lsk_640_640,0.756,0.569,0.338,0.529,0.656,3.130,8.288,8.2,125.0
att_yolov8n_eca_640_640,0.753,0.576,0.338,0.535,0.658,3.011,8.196,8.0,131.0
att_yolov8n_gam_640_640,0.759,0.583,0.342,0.541,0.662,4.651,9.506,8.0,130.9
att_yolov8n_se_640_640,0.751,0.584,0.353,0.544,0.659,3.019,8.201,8.2,127.3
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
att_yolov8n_cpca_640_640,0.760,0.588,0.358,0.549,0.667,3.138,8.425,8.1,126.7
att_yolov8n_lska_640_640,0.751,0.590,0.366,0.552,0.661,3.089,8.256,8.2,132.3


att_yolov8n_lska_640_640相较于base_yolov8n_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.756,135.0
map,0.751 (-00.4%),0.590 (+00.7%),0.366 (+05.2%),0.552 (+01.3%),0.661 (-00.2%),3.089 (+02.6%),8.256 (+00.8%),8.2 (+05.4%),132.3 (-02.0%)


## FPN模块

In [16]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|fpn_yolov8n')]
metrics = metrics[~metrics.index.str.contains('merge')]
display_metircs(metrics, filename='fpn')
display_single_improve(metrics, 'base_yolov8n_640_640', 'fpn_yolov8n_bifpn_640_640', filename='improve_bifpn_vs_base')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
fpn_yolov8n_afpn_640_640,0.761,0.574,0.333,0.532,0.660,4.667,15.641,10.2,101.9
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
fpn_yolov8n_bifpn_640_640,0.758,0.591,0.365,0.553,0.666,3.027,8.247,7.8,135.4


fpn_yolov8n_bifpn_640_640相较于base_yolov8n_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.756,135.0
map,0.758 (+00.5%),0.591 (+00.9%),0.365 (+04.9%),0.553 (+01.5%),0.666 (+00.6%),3.027 (+00.5%),8.247 (+00.6%),7.8 (+00.2%),135.4 (+00.3%)


## IOU

In [17]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|iou_yolov8n') & ~metrics.index.str.contains('merge')]
display_metircs(metrics, filename='iou')
# todo 评估不同iou对应的后处理时间

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
iou_yolov8n_NWD_640_640,0.705,0.518,0.334,0.487,0.607,3.011,8.194,7.9,136.0
iou_yolov8n_EIoU_640_640,0.757,0.572,0.338,0.532,0.658,3.011,8.194,8.0,132.4
iou_yolov8n_WIoU_640_640,0.742,0.577,0.343,0.538,0.652,3.011,8.194,7.9,135.5
iou_yolov8n_ShapeIoU_640_640,0.761,0.581,0.369,0.545,0.666,3.011,8.194,7.6,140.8
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
iou_yolov8n_MDPIoU1_640_640,0.746,0.589,0.357,0.549,0.660,3.011,8.194,7.5,139.5
iou_yolov8n_GIoU_640_640,0.765,0.588,0.371,0.551,0.671,3.011,8.194,7.5,141.3
iou_yolov8n_MDPIoU2_640_640,0.759,0.593,0.380,0.557,0.668,3.011,8.194,7.4,142.6
iou_yolov8n_DIoU_640_640,0.765,0.594,0.383,0.558,0.669,3.011,8.194,7.9,132.6


## 上采样

In [18]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|up_yolov8n') & ~metrics.index.str.contains('merge')]
display_metircs(metrics, filename='up')

/home/yeli/anaconda3/envs/lard/lib/python3.9/site-packages/pandas/io/formats/style.py:3851: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
up_yolov8n_carafe_640_640,0.754,0.576,0.358,0.539,0.659,3.083,8.373,8.8,121.0
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0


## 数据增强

In [19]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|aug_yolov8n') & ~metrics.index.str.contains('merge')]
display_metircs(metrics, filename='aug')
display_single_improve(metrics, 'base_yolov8n_640_640', 'aug_yolov8n_aug_all_5_640_640', filename='improve_aug5_vs_base')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
aug_yolov8n_aug_all_1_640_640,0.746,0.596,0.368,0.558,0.661,3.011,8.194,8.1,129.1
aug_yolov8n_aug_all_20_640_640,0.755,0.591,0.417,0.561,0.666,3.011,8.194,7.8,135.4
aug_yolov8n_aug_all_5_640_640,0.749,0.600,0.381,0.563,0.664,3.011,8.194,7.5,140.2
aug_yolov8n_aug_all_15_640_640,0.750,0.596,0.408,0.565,0.663,3.011,8.194,7.6,139.4
aug_yolov8n_aug_all_10_640_640,0.756,0.603,0.412,0.572,0.671,3.011,8.194,7.9,132.4


aug_yolov8n_aug_all_5_640_640相较于base_yolov8n_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.756,135.0
map,0.749 (-00.7%),0.600 (+02.4%),0.381 (+09.5%),0.563 (+03.3%),0.664 (+00.3%),3.011 (+00.0%),8.194 (+00.0%),7.5 (-03.8%),140.2 (+03.9%)


# 组合模块分析

## ATT+FPN

In [20]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|att_fpn_yolov8n')]
display_metircs(metrics, filename='att_fpn')
# display_single_mprove(metrics, 'base_yolov8n_640_640', 'merge_att_fpn_yolov8n_lska_bifpn_640_640')


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
merge_att_fpn_yolov8n_ese_bifpn_640_640,0.755,0.572,0.322,0.530,0.655,nan,nan,nan,nan
merge_att_fpn_yolov8n_ta_bifpn_640_640,0.757,0.575,0.338,0.534,0.658,nan,nan,nan,nan
merge_att_fpn_yolov8n_sa_bifpn_640_640,0.757,0.575,0.350,0.537,0.661,nan,nan,nan,nan
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
merge_att_fpn_yolov8n_cpca_bifpn_640_640,0.763,0.589,0.368,0.552,0.670,nan,nan,nan,nan
merge_att_fpn_yolov8n_lska_bifpn_640_640,0.761,0.598,0.374,0.559,0.672,3.106,8.308,8.0,129.3


## FPN+IOU

In [21]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|merge_fpn_iou_yolov8n')]
display_metircs(metrics, filename='fpn_iou')
# display_improve(metrics, 'base_yolov8n_640_640', 'merge_att_fpn_yolov8n_lska_bifpn_640_640')


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
merge_fpn_iou_yolov8n_bifpn_EIoU_640_640,0.755,0.575,0.359,0.537,0.659,3.027,8.247,8.5,123.3
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0


## ATT+FPN+IOU

In [22]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|merge_att_fpn_yolov8n_lska_bifpn_640_640|att_fpn_iou_yolov8n|att_fpn_iou_aug_yolov8n')]
metrics = metrics[~metrics.index.str.contains('aug')]
display_metircs(metrics, filename='att_fpn_iou')
print('可以用此处的实验 验证EIOU有效')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
merge_att_fpn_iou_yolov8n_lska_bifpn_NWD_640_640,0.696,0.505,0.317,0.473,0.594,3.106,8.308,8.6,122.9
merge_att_fpn_iou_yolov8n_lska_bifpn_WIoU_640_640,0.751,0.574,0.365,0.539,0.658,3.106,8.308,8.3,126.5
merge_att_fpn_iou_yolov8n_lska_bifpn_MDPIoU1_640_640,0.743,0.581,0.347,0.541,0.653,3.106,8.308,8.3,126.4
merge_att_fpn_iou_yolov8n_lska_bifpn_SIoU_640_640,0.762,0.579,0.357,0.541,0.662,3.106,8.308,8.1,129.8
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
merge_att_fpn_iou_yolov8n_lska_bifpn_ShapeIoU_640_640,0.757,0.587,0.360,0.548,0.664,3.106,8.308,7.8,135.2
merge_att_fpn_iou_yolov8n_lska_bifpn_MDPIoU2_640_640,0.762,0.594,0.357,0.555,0.670,3.106,8.308,8.2,127.7
merge_att_fpn_iou_yolov8n_lska_bifpn_DIoU_640_640,0.764,0.595,0.365,0.556,0.671,3.106,8.308,7.8,135.1
merge_att_fpn_iou_yolov8n_lska_bifpn_GIoU_640_640,0.751,0.591,0.378,0.556,0.665,3.106,8.308,8.1,129.4


可以用此处的实验 验证EIOU有效


# 提出模型

## 消融

In [24]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics, sort=False)
list_model = [
    # baseline
    'base_yolov8n_640_640',

    # one
    'att_yolov8n_lska_640_640', 
    'fpn_yolov8n_bifpn_640_640', 
    'iou_yolov8n_EIoU_640_640',

    # two
    'merge_att_fpn_yolov8n_lska_bifpn_640_640', 
    'merge_att_iou_yolov8n_lska_EIoU_640_640',
    'merge_fpn_iou_yolov8n_bifpn_EIoU_640_640',

    # three
    'merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU_640_640',

    # four
    'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640',
]

metrics = metrics.loc[list_model]
display_metircs(metrics, filename='ablation')
display_single_improve(metrics, 'base_yolov8n_640_640', 'merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU_640_640', filename='improve_proposed_woeda_vs_base')
display_single_improve(metrics, 'base_yolov8n_640_640', 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640', filename='improve_proposed_vs_base')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
att_yolov8n_lska_640_640,0.751,0.590,0.366,0.552,0.661,3.089,8.256,8.2,132.3
fpn_yolov8n_bifpn_640_640,0.758,0.591,0.365,0.553,0.666,3.027,8.247,7.8,135.4
iou_yolov8n_EIoU_640_640,0.757,0.572,0.338,0.532,0.658,3.011,8.194,8.0,132.4
merge_att_fpn_yolov8n_lska_bifpn_640_640,0.761,0.598,0.374,0.559,0.672,3.106,8.308,8.0,129.3
merge_att_iou_yolov8n_lska_EIoU_640_640,0.756,0.576,0.339,0.536,0.657,3.089,8.256,7.7,135.8
merge_fpn_iou_yolov8n_bifpn_EIoU_640_640,0.755,0.575,0.359,0.537,0.659,3.027,8.247,8.5,123.3
merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU_640_640,0.762,0.600,0.393,0.563,0.675,3.106,8.308,7.9,131.4
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,0.760,0.611,0.413,0.578,0.678,3.106,8.308,8.4,125.3


merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU_640_640相较于base_yolov8n_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.756,135.0
map,0.762 (+01.1%),0.600 (+02.4%),0.393 (+12.9%),0.563 (+03.3%),0.675 (+02.0%),3.106 (+03.2%),8.308 (+01.4%),7.9 (+02.3%),131.4 (-02.7%)


merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640相较于base_yolov8n_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.756,135.0
map,0.760 (+00.8%),0.611 (+04.3%),0.413 (+18.7%),0.578 (+06.1%),0.678 (+02.4%),3.106 (+03.2%),8.308 (+01.4%),8.4 (+08.2%),125.3 (-07.2%)


## 提出模型+AUG+尺寸

In [25]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics)
aug_prob = '5'
metrics = metrics[metrics.index.str.contains(f'base_yolov8n_640_640|merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU|merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_{aug_prob}_')]
display_metircs(metrics, filename='proposed_aug')
display_single_improve(metrics, 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640', 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_1280', filename='improve_proposed_640_1280_vs_proposed')
display_single_improve(metrics, 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640', 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_1280_1280', filename='improve_proposed_1280_1280_vs_proposed')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU_640_640,0.762,0.600,0.393,0.563,0.675,3.106,8.308,7.9,131.4
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640_fp16,0.757,0.610,0.413,0.578,0.676,3.106,8.308,8.0,130.3
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,0.760,0.611,0.413,0.578,0.678,3.106,8.308,8.4,125.3
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_1280_fp16,0.783,0.740,0.521,0.704,0.745,3.106,33.232,13.5,83.5
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_1280,0.785,0.744,0.517,0.707,0.747,3.106,33.232,12.7,89.3
merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_1280_1280,0.848,0.750,0.489,0.709,0.786,3.106,33.232,12.7,90.9


merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_1280相较于merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.760,0.611,0.413,0.578,0.678,3.106,8.308,8.394,125.3
map,0.785 (+03.3%),0.744 (+21.8%),0.517 (+25.2%),0.707 (+22.3%),0.747 (+10.2%),3.106 (+00.0%),33.232 (+300.0%),12.7 (+50.8%),89.3 (-28.7%)


merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_1280_1280相较于merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640的改进


,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
map,0.760,0.611,0.413,0.578,0.678,3.106,8.308,8.394,125.3
map,0.848 (+11.6%),0.750 (+22.7%),0.489 (+18.4%),0.709 (+22.7%),0.786 (+15.9%),3.106 (+00.0%),33.232 (+300.0%),12.7 (+51.2%),90.9 (-27.5%)


## 提升计算

In [26]:
basemodel = 'base_yolov8n_640_640'
proposed_640_640 = 'merge_att_fpn_iou_yolov8n_lska_bifpn_EIoU_640_640'
proposed_aug5_640_640 = 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640'
proposed_aug5_640_1280 = 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_640_1280'
proposed_aug5_1280_1280 = 'merge_att_fpn_iou_aug_yolov8n_lska_bifpn_EIoU_aug_all_5_1280_1280'

display_sml_improve(basemodel, proposed_640_640, filename='improve_proposed_640_640_vs_base')
display_sml_improve(basemodel, proposed_aug5_640_640, filename='improve_proposed_5_640_640_vs_base')
display_sml_improve(basemodel, proposed_aug5_640_1280, filename='improve_proposed_5_640_1280_vs_base')
display_sml_improve(basemodel, proposed_aug5_1280_1280, filename='improve_proposed_5_1280_1280_vs_base')

,test_synth,test_real_nominal,test_real_edge,test_real,test
metrics,,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.188,0.110,0.174,0.170
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.621,0.461,0.268,0.432,0.529
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.813,0.784,0.439,0.715,0.777
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.193 (+02.7%),0.149 (+35.5%),0.185 (+06.3%),0.184 (+08.2%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.632 (+01.8%),0.473 (+02.6%),0.319 (+19.0%),0.450 (+04.2%),0.542 (+02.5%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.821 (+01.0%),0.799 (+01.9%),0.478 (+08.9%),0.735 (+02.8%),0.788 (+01.4%)


,test_synth,test_real_nominal,test_real_edge,test_real,test
metrics,,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.188,0.110,0.174,0.170
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.621,0.461,0.268,0.432,0.529
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.813,0.784,0.439,0.715,0.777
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.205 (+09.0%),0.191 (+73.6%),0.201 (+15.5%),0.202 (+18.8%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.634 (+02.1%),0.476 (+03.3%),0.306 (+14.2%),0.452 (+04.6%),0.546 (+03.2%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.817 (+00.5%),0.815 (+04.0%),0.521 (+18.7%),0.759 (+06.2%),0.794 (+02.2%)


,test_synth,test_real_nominal,test_real_edge,test_real,test
metrics,,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.188,0.110,0.174,0.170
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.621,0.461,0.268,0.432,0.529
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.813,0.784,0.439,0.715,0.777
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.403 (+114.4%),0.304 (+176.4%),0.388 (+123.0%),0.388 (+128.2%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.702 (+13.0%),0.668 (+44.9%),0.437 (+63.1%),0.636 (+47.2%),0.664 (+25.5%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.825 (+01.5%),0.879 (+12.1%),0.604 (+37.6%),0.827 (+15.7%),0.823 (+05.9%)


,test_synth,test_real_nominal,test_real_edge,test_real,test
metrics,,,,,
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.188,0.110,0.174,0.170
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.621,0.461,0.268,0.432,0.529
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.813,0.784,0.439,0.715,0.777
"AP (IoU=0.50:0.95, area=small, maxDets=100)",-,0.404 (+114.9%),0.307 (+179.1%),0.387 (+122.4%),0.387 (+127.6%)
"AP (IoU=0.50:0.95, area=medium, maxDets=100)",0.760 (+22.4%),0.676 (+46.6%),0.424 (+58.2%),0.642 (+48.6%),0.701 (+32.5%)
"AP (IoU=0.50:0.95, area=large, maxDets=100)",0.887 (+09.1%),0.890 (+13.5%),0.576 (+31.2%),0.832 (+16.4%),0.867 (+11.6%)


# 微调分析

## 5%增强

In [27]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics, is_finetune=True)
aug_prob = '5'
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|finetune')]
metrics = metrics[metrics.index.str.contains(f'base_yolov8n_640_640|EIoU_aug_all_{aug_prob}')]
display_metircs(metrics, 'base_yolov8n_640_640', is_finetune=True, filename=f'finetune_{aug_prob}')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
finetune_single_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,0.712,0.579,0.362,0.542,0.635,nan,nan,nan,nan
finetune_double_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,nan,0.615,0.381,0.576,nan,nan,nan,nan,nan
finetune_triple_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,nan,nan,0.557,nan,nan,nan,nan,nan,nan
finetune_triple_split_yolov8n_lska_bifpn_EIoU_aug_all_5_640_640,nan,nan,0.579,nan,nan,nan,nan,nan,nan


## 10%增强

In [28]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics, is_finetune=True)
aug_prob = '10'
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|finetune')]
metrics = metrics[metrics.index.str.contains(f'base_yolov8n_640_640|EIoU_aug_all_{aug_prob}')]
display_metircs(metrics, 'base_yolov8n_640_640', is_finetune=True, filename=f'finetune_{aug_prob}')
print('每一个metric都提高了 相较于5')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
finetune_single_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640,0.722,0.584,0.376,0.549,0.645,nan,nan,nan,nan
finetune_double_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640,nan,0.627,0.401,0.590,nan,nan,nan,nan,nan
finetune_triple_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640,nan,nan,0.562,nan,nan,nan,nan,nan,nan
finetune_triple_split_yolov8n_lska_bifpn_EIoU_aug_all_10_640_640,nan,nan,0.582,nan,nan,nan,nan,nan,nan


每一个metric都提高了 相较于5


## 20%增强

In [29]:
metrics = read_all_single_map('results/tables/metrics')
metrics = set_metrics(metrics, is_finetune=True)
aug_prob = '20'
metrics = metrics[metrics.index.str.contains('base_yolov8n_640_640|finetune')]
metrics = metrics[metrics.index.str.contains(f'base_yolov8n_640_640|EIoU_aug_all_{aug_prob}')]
display_metircs(metrics, 'base_yolov8n_640_640', is_finetune=True, filename=f'finetune_{aug_prob}')
print('只有real_edge增强了 相较于10%')

,test_synth,test_real_nominal,test_real_edge,test_real,test,param,flops,delay,fps
models,,,,,,,,,
base_yolov8n_640_640,0.754,0.586,0.348,0.545,0.662,3.011,8.194,7.8,135.0
finetune_single_yolov8n_lska_bifpn_EIoU_aug_all_20_640_640,0.702,0.573,0.371,0.539,0.628,nan,nan,nan,nan
finetune_double_yolov8n_lska_bifpn_EIoU_aug_all_20_640_640,nan,0.624,0.412,0.589,nan,nan,nan,nan,nan
finetune_triple_yolov8n_lska_bifpn_EIoU_aug_all_20_640_640,nan,nan,0.586,nan,nan,nan,nan,nan,nan
finetune_triple_split_yolov8n_lska_bifpn_EIoU_aug_all_20_640_640,nan,nan,0.589,nan,nan,nan,nan,nan,nan


只有real_edge增强了 相较于10%
